In [1]:
using ElectricGrid
using Interpolations

using PlotlyJS
# using .LookupTables

WebIO._IJuliaInit()

In [2]:
using BenchmarkTools

In [3]:
x = collect(0:0.1:1)
y = vec([293.15 313.15])
grid = [0.010342 0.0012244;
0.0067316 0.0011396;
0.0051156 0.0012661;
0.0043447 0.0012265;
0.0038826 0.0011163;
0.0034226 0.0009968;
0.003346 0.0011458;
0.0033222 0.001345;
0.0033201 0.0013091;
0.0032886 0.0010986;
0.0028114 0.0010309];

In [4]:
@benchmark itp = LinearInterpolation((x,y), grid)

BenchmarkTools.Trial: 10000 samples with 178 evaluations.
 Range (min … max):  608.427 ns … 224.287 μs  ┊ GC (min … max):  0.00% … 99.38%
 Time  (median):       1.067 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.323 μs ±   5.948 μs  ┊ GC (mean ± σ):  17.19% ±  3.83%

   ▄▂        █▆▃▃                                                
  ▇██▆▃▂▂▂▂▆██████▅▄▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  608 ns           Histogram: frequency by time         2.93 μs <

 Memory estimate: 1.88 KiB, allocs estimate: 16.

In [5]:
itp = LinearInterpolation((x,y), grid)

11×2 extrapolate(interpolate((::Vector{Float64},::Vector{Float64}), ::Matrix{Float64}, Gridded(Linear())), Throw()) with element type Float64:
 0.010342   0.0012244
 0.0067316  0.0011396
 0.0051156  0.0012661
 0.0043447  0.0012265
 0.0038826  0.0011163
 0.0034226  0.0009968
 0.003346   0.0011458
 0.0033222  0.001345
 0.0033201  0.0013091
 0.0032886  0.0010986
 0.0028114  0.0010309

In [6]:
@benchmark itp(0.5, 300)

BenchmarkTools.Trial: 10000 samples with 995 evaluations.
 Range (min … max):  32.764 ns … 117.889 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     37.286 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   38.261 ns ±   3.928 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▁▁▁▁▂▂▃▆███▇▅▅▄▄▃▂▂▃▃▃▃▂▃▂▂▂▁▁▁▁ ▁                         ▃
  ▇█████████████████████████████████▇█▇▆█▇▆▆▆▅▅▅▅▅▆▆▅▅▅▄▅▆▆▅▁▃ █
  32.8 ns       Histogram: log(frequency) by time      54.8 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

## Define Battery moduls

In [7]:
Base.@kwdef mutable struct battery_module
     R = [0.010342 0.0012244;
        0.0067316 0.0011396;
        0.0051156 0.0012661;
        0.0043447 0.0012265;
        0.0038826 0.0011163;
        0.0034226 0.0009968;
        0.003346 0.0011458;
        0.0033222 0.001345;
        0.0033201 0.0013091;
        0.0032886 0.0010986;
        0.0028114 0.0010309]     # Resistance for n-th RC pair
#     R = [0.02 0.04;
#         0.04 0.08;
#         0.06 0.12;
#         0.08 0.16;
#         0.10 0.20;
#         0.12 0.24;
#         0.14 0.28;
#         0.16 0.32;
#         0.18 0.36;
#         0.2 0.40;
#         0.22 0.44]
    C = [2287.7 11897;
        6122 24515;
        18460 42098;
        20975 44453;
        15254 33098;
        10440 24492;
        13903 32975;
        16694 40007;
        15784 35937;
        12165 26430;
        9118 24795]   # Capacitance for n-th RC pair
#     C = R*10000
end;

In [8]:
Base.@kwdef mutable struct battery_block
    battery_module::battery_module
    T = 273.15    # temperature offset
    n = 3                      # number of RC pairs
    R_0 = 0.016                  # series resistance
    V_0 = [3.5042 3.5136;
        3.5573 3.5646;
        3.6009 3.6153;
        3.6393 3.6565;
        3.6742 3.6889;
        3.7121 3.7214;
        3.7937 3.8078;
        3.8753 3.8945;
        3.97 3.9859;
        4.0764 4.0821;
        4.1924 4.193]          # Battery open-circuit voltage
    Q = 0                      # Current Ladung in A*s
    Q_0 = n*26*3600              # Nominal Ladung
    SOC = 0                    # State of Charge (SOC)
    SOC_BP = vec([0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1])
    T_BP = vec([293.15 313.15])
    tau = 1 #0.1e-3
    mode = nothing
    V = 0
    LT_V0 = LinearInterpolation((SOC_BP,T_BP), V_0)
    LT_R = LinearInterpolation((SOC_BP,T_BP), battery_module.R)
    I_limit = 15
    P_1h = nothing
end;

In [9]:
function update(self::battery_block, I_batt, T)
    
    T_ = self.T + T
    
    self.Q = clamp(self.Q - I_batt * self.tau, 0, self.Q_0) # Clamp 
    self.SOC = clamp(self.Q/self.Q_0, 0, 1)
    
    if sign(I_batt) == -1
        self.mode = "charge"
    else
        self.mode = "discharge"
    end
    
    P_1h = self.Q/3600 * self.LT_V0(self.SOC, T_)
    
    I_batt = clamp(I_batt, -self.I_limit, self.I_limit) # Current limitation
    
    self.V = self.LT_V0(self.SOC, T_) - self.n * self.LT_R(self.SOC, T_) * I_batt - self.R_0 * I_batt 
    
end

update (generic function with 1 method)

In [10]:
Cell = battery_module();

In [11]:
Battery = battery_block(battery_module=Cell);

### Test

In [12]:
SOC_load = []
Voltage_load = []
Current_load = []

SOC_discharge = []
Voltage_discharge = []
Current_discharge = []

Any[]

In [13]:
Ts = 1e-4
I_s = .05

0.05

In [14]:
for i in 1:1000000*6
    I_in = -I_s

    update(Battery, I_in, 20)

    append!(SOC_load, Battery.SOC)
    append!(Current_load, I_in)
    append!(Voltage_load, Battery.V)
    
    # println("Mode: $(Battery.mode)")
    # println("State of Charge $(Battery.SOC)")
    # println("Output Voltage $(Battery.V)")
end

for i in 1:1000000*6
    I_in = I_s

    update(Battery, I_in, 20)

    append!(SOC_discharge, Battery.SOC)
    append!(Current_discharge, I_in)
    append!(Voltage_discharge, Battery.V)
    
end

In [15]:
size(SOC_load)

(6000000,)

In [16]:
size(SOC_load[1:1000:end])

(6000,)

In [17]:
plot([scatter(x=SOC_load[1:100000:end], y=Voltage_load[1:100000:end], mode="markers", name="Charge"),
scatter(x=SOC_discharge[1:100000:end], y=Voltage_discharge[1:100000:end], mode="markers", name="Discharge")])

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin and template"

In [ ]:
plot([scatter(x=SOC_load, y=Current_load, mode="markers", name="Charge"),
      scatter(x=SOC_discharge, y=Current_discharge, mode="markers", name="Discharge")])